# Pruebas con Optuna

In [1]:
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, TensorDataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import os
import sys

# get the directory path of the file
dir_path = os.getcwd()

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from plot_functions import *
from Physics_Loss import *
from utils import *
sys.path.append('../Convolutional_NN')

from Dataset_Class import *

from architectures.convlstm import *

In [2]:
epochs = 5
n_train = 10
n_val = 2
sequence_length = 2

In [4]:
# Al principio del notebook
dataset_train = load_trimmed_dataset(base_path=dir_path, dataset_type='train', max_samples=n_train, time_steps_output=sequence_length)
dataset_val = load_trimmed_dataset(base_path=dir_path, dataset_type='val', max_samples=n_val, time_steps_output=sequence_length)

input_train, output_train = prepare_data_for_convlstm(dataset_train, device=device)
input_val, output_val = prepare_data_for_convlstm(dataset_val, device=device)

def get_data_loaders_from_tensors(batch_size):
    train_loader = DataLoader(TensorDataset(input_train, output_train), batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(TensorDataset(input_val, output_val), batch_size=batch_size, shuffle=False)
    return train_loader, val_loader

✅ Cargando dataset train desde: c:\Users\ismael.gallo\Desktop\ia_thermal\ismaelgallo\datasets\PCB_transient_dataset_train.pth


c:\Users\ismael.gallo\Desktop\ia_thermal\ismaelgallo\../Convolutional_NN\Dataset_Class.py:299: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  base_dataset = torch.load(full_p

✅ Cargando dataset val desde: c:\Users\ismael.gallo\Desktop\ia_thermal\ismaelgallo\datasets\PCB_transient_dataset_val.pth


## ConvLSTM

In [5]:
class ConvLSTMWrapper(nn.Module):
    def __init__(self, input_dim, hidden_dim, kernel_size, num_layers):
        super().__init__()
        self.convlstm = ConvLSTM(
            input_dim=input_dim,
            hidden_dim=hidden_dim,
            kernel_size=kernel_size,
            num_layers=num_layers,
            batch_first=True,
            bias=True,
            return_all_layers=False
        )

    def forward(self, x):
        output, _ = self.convlstm(x)  # output es lista [layer_output]
        return output[0]              # devolvemos directamente el tensor (B, T, C, H, W)


In [6]:
def objective(trial):
    # Hiperparámetros
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32])
    hidden_dim_val = trial.suggest_categorical("hidden_dim", [16, 32, 64])
    kernel_size_val = trial.suggest_categorical("kernel_size", [3, 5])
    num_layers = trial.suggest_int("num_layers", 1, 4)

    # Adaptar al formato requerido por ConvLSTM
    hidden_dim = [hidden_dim_val] * num_layers
    kernel_size = [(kernel_size_val, kernel_size_val)] * num_layers

    # Crear modelo ConvLSTM
    model = ConvLSTMWrapper(
        input_dim=3,  # o el número de canales reales
        hidden_dim=hidden_dim,
        kernel_size=kernel_size,
        num_layers=num_layers
    ).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
    criterion = torch.nn.MSELoss()

    # Obtener dataloaders (train y val)
    train_loader, val_loader = get_data_loaders_from_tensors(batch_size=batch_size)

    best_val_loss = float('inf')
    for epoch in range(epochs):
        train_loss = train_one_epoch(model, train_loader, criterion, optimizer, device, epoch, epochs)
        val_loss = evaluate(model, val_loader, criterion, device)
        scheduler.step(val_loss)

        if val_loss < best_val_loss:
            best_val_loss = val_loss

        trial.report(val_loss, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return best_val_loss


In [7]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)


[I 2025-05-05 13:20:19,344] A new study created in memory with name: no-name-c60eb86e-f39d-43eb-ac4e-910656e9a367
Epoch 1/5 - Training:   0%|          | 0/1 [00:00<?, ?it/s]c:\Users\ismael.gallo\anaconda3\envs\ismael_minimal\lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([10, 2, 1, 13, 13])) that is different to the input size (torch.Size([10, 2, 32, 13, 13])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\ismael.gallo\anaconda3\envs\ismael_minimal\lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([2, 2, 1, 13, 13])) that is different to the input size (torch.Size([2, 2, 32, 13, 13])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
[I 2025-05-05 13:20:22,309]

In [8]:
print("Best trial:")
for key, val in study.best_trial.params.items():
    print(f"{key}: {val}")

Best trial:
lr: 0.008646266608674288
batch_size: 32
hidden_dim: 64
kernel_size: 5
num_layers: 4


In [9]:
optuna.visualization.plot_optimization_history(study).show()
optuna.visualization.plot_param_importances(study).show()
